In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ''
import tensorflow as tf
print("Num GPUs available: ",len(tf.config.experimental.list_physical_devices('GPU')))
# print (tf.__version__)

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
from tensorflow.keras.backend import clear_session
clear_session()

In [ ]:
import keras
import tensorflow as tf
import pandas as pd
from keras import layers
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

strategy = tf.distribute.MirroredStrategy()

In [ ]:
with strategy.scope():
    #Dataset Preparation
    #Loading MIAS, DDSM, INBreast, and VinDR Datasets
    mias_df = pd.read_csv('D:/MIAS_all-mias/FINAL_mias.csv')
    print ("MIAS Dataset")
    print (mias_df.info())
    print ("Number of records: ",len(mias_df['labels']))
    print (mias_df['labels'].value_counts())
    print ("--------------------------------------------")
    
    inbreast_df = pd.read_csv("D:/InBreast/2023/InBreast/FINAL_inb.csv")
    print ("INbreast Dataset")
    print (inbreast_df.info())
    print ("Number of records: ",len(inbreast_df['labels']))
    print (inbreast_df['labels'].value_counts())
    print ("--------------------------------------------")
    
    ddsm_df = pd.read_csv("D:/CBIS-DDSM/FINAL_ddsm.csv")
    print ("DDSM Dataset")
    print (ddsm_df.info())
    print ("Number of records: ",len(ddsm_df['labels']))
    print (ddsm_df['labels'].value_counts())
    print ("--------------------------------------------")
    
    vin_df = pd.read_csv("D:/VinDRmammo/vindr/vindr/FINAL_vin.csv")
    print ("VinDR-Mammo Dataset")
    print (vin_df.info())
    print ("Number of records: ",len(vin_df['labels']))
    print (vin_df['labels'].value_counts())
    print ("--------------------------------------------")
    
    # List of DataFrames to concatenate
    dfs = [mias_df, inbreast_df, ddsm_df, vin_df]
    
    # Concatenate the DataFrames
    merged_df = pd.concat(dfs, ignore_index=True)
    
    # Display information about the merged dataset
    print("Merged Dataset")
    print(merged_df.info())
    print("Number of records: ", len(merged_df['labels']))
    print(merged_df['labels'].value_counts())
    print ("------------------------------------------------")
    print ("Sample Image Path: ",merged_df['image_path'][0])
    print ("Sample label: ", merged_df['labels'][0])
    print ('###################################################')


    # Step 1: Split into training (80%) and temp (20%)
    train_df, temp_df = train_test_split(merged_df, test_size=0.2, random_state=42, stratify=merged_df['labels'])
    
    # Step 2: Split the temp into validation (75% of 20% -> 15% of total) and test (25% of 20% -> 5% of total)
    valid_df, test_df = train_test_split(temp_df, test_size=0.25, random_state=42, stratify=temp_df['labels'])
    
    print(f"Training set size: {len(train_df)}")
    print(f"Validation set size: {len(valid_df)}")
    print(f"Test set size: {len(test_df)}")

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Directly use integers for classes
class_labels = [0, 1]  # Corresponding to 'BENIGN': 0, 'MALIGNANT': 1

# Convert string labels to integers
train_labels_int = train_df['labels'].map({'BENIGN': 0, 'MALIGNANT': 1}).values

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=class_labels,
                                     y=train_labels_int)

# Create a dictionary mapping class indices to their respective weights
class_weights_dict = {class_label: weight for class_label, weight in zip(class_labels, class_weights)}

print("Class Weights Dictionary:", class_weights_dict)

In [ ]:
positional_emb = True
conv_layers = 2
projection_dim = 128

num_heads = 2
transformer_units = [
    projection_dim,
    projection_dim,
]
transformer_layers = 2
stochastic_depth_rate = 0.1

learning_rate = 0.001
weight_decay = 0.0001
batch_size = 16
num_epochs = 30
image_size = 320
input_shape = (image_size, image_size, 3)

In [ ]:
with strategy.scope():
    def preprocess_image(image_path, label):
        # Read the image from file
        image = tf.io.read_file(image_path)
        
        # Dynamically decode image based on extension
        def decode_jpeg(image):
            return tf.image.decode_jpeg(image, channels=3)
        
        def decode_png(image):
            return tf.image.decode_png(image, channels=3)
        
        # Use tf.cond to dynamically decide how to decode images
        image = tf.cond(
            tf.strings.regex_full_match(image_path, ".+\.jpg"),
            lambda: decode_jpeg(image),
            lambda: decode_png(image)
        )
        image = tf.image.resize(image, [image_size, image_size])
        image = tf.cast(image, tf.float32) / 255.0  # Normalize image pixels to [0, 1]
        
        return image, label
    
    
    def prepare_dataset(df, batch_size=32):
        # Convert labels from string to integers (0 or 1)
        labels = df['labels'].map({'BENIGN': 0, 'MALIGNANT': 1}).values
        # Create a dataset from the image paths and labels
        paths = df['image_path'].values
        dataset = tf.data.Dataset.from_tensor_slices((paths, labels))
        # Map the preprocessing function to each element
        dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
        # Shuffle the dataset (Note: Only shuffle the training dataset)
        # dataset = dataset.shuffle(buffer_size=len(df))
        # Batch the dataset
        dataset = dataset.batch(batch_size)
        # Prefetch to improve speed of input pipeline
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        return dataset
    
    # Prepare the datasets
    train_dataset = prepare_dataset(train_df, batch_size=batch_size)
    valid_dataset = prepare_dataset(valid_df, batch_size=batch_size)
    test_dataset = prepare_dataset(test_df, batch_size=batch_size)
    
    # Take one batch from the training dataset
    for images, labels in train_dataset.take(1):
        # Take the first image from this batch
        sample_image = images[0]
        sample_label = labels[0].numpy()
        
        # Plot the sample image
        plt.figure(figsize=(2, 2))
        plt.imshow(sample_image)
        plt.title(f"Label: {sample_label}")
        plt.axis("off")
        plt.show()
    
    def print_dataset_shapes(dataset, dataset_name):
        for images, labels in dataset.take(1):  # Only take a single batch
            print(f"{dataset_name} - Images shape: {images.shape}, Labels shape: {labels.shape}")
    
    print_dataset_shapes(train_dataset, "Train")
    print_dataset_shapes(valid_dataset, "Valid")
    print_dataset_shapes(test_dataset, "Test")

In [ ]:
with strategy.scope():
    class CCTTokenizer(layers.Layer):
        def __init__(
            self,
            kernel_size=3,
            stride=1,
            padding=1,
            pooling_kernel_size=3,
            pooling_stride=2,
            num_conv_layers=conv_layers,
            num_output_channels=[64, 128],
            positional_emb=positional_emb,
            **kwargs,
        ):
            super().__init__(**kwargs)
    
            # Debugging: print the initialization parameters
            print("Initializing CCTTokenizer with parameters:")
            print("  kernel_size:", kernel_size)
            print("  stride:", stride)
            print("  padding:", padding)
            print("  pooling_kernel_size:", pooling_kernel_size)
            print("  pooling_stride:", pooling_stride)
            print("  num_conv_layers:", num_conv_layers)
            print("  num_output_channels:", num_output_channels)
    
            self.conv_model = keras.Sequential()
            for i in range(num_conv_layers):
                self.conv_model.add(
                    layers.Conv2D(
                        num_output_channels[i],
                        kernel_size,
                        stride,
                        padding="valid",
                        use_bias=False,
                        activation="relu",
                        kernel_initializer="he_normal",
                    )
                )
                self.conv_model.add(layers.ZeroPadding2D(padding))
                self.conv_model.add(
                    layers.MaxPooling2D(pooling_kernel_size, pooling_stride, "same")
                )
    
            self.positional_emb = positional_emb
    
        def call(self, images):
            # Debugging: print the shape of the input images
            print("Input shape/CCTTokenizer:", images.shape)
    
            outputs = self.conv_model(images)
    
            # Debugging: print the shape of the output after the convolutional model
            print("Output shape after conv_model:", outputs.shape)
    
            reshaped = tf.reshape(
                outputs,
                (
                    -1,
                    tf.shape(outputs)[1] * tf.shape(outputs)[2],
                    tf.shape(outputs)[-1],
                ),
            )
    
            # Debugging: print the shape after reshaping
            print("Output shape after reshaping/CCTTokenizer:", reshaped.shape)
    
            return reshaped

In [ ]:
with strategy.scope():
    class PositionEmbedding(keras.layers.Layer):
        def __init__(self, sequence_length, initializer="glorot_uniform", **kwargs):
            super().__init__(**kwargs)
            if sequence_length is None:
                raise ValueError("`sequence_length` must be an Integer, received `None`.")
            self.sequence_length = int(sequence_length)
            self.initializer = keras.initializers.get(initializer)
    
        def get_config(self):
            config = super().get_config()
            config.update({
                "sequence_length": self.sequence_length,
                "initializer": keras.initializers.serialize(self.initializer),
            })
            return config
    
        def build(self, input_shape):
            feature_size = input_shape[-1]
            self.position_embeddings = self.add_weight(
                name="embeddings",
                shape=[self.sequence_length, feature_size],
                initializer=self.initializer,
                trainable=True,
            )
    
        def call(self, inputs, start_index=0):
            print("Input shape/Positional Embedding:", inputs.shape)
            shape = tf.shape(inputs)
            feature_length = shape[-1]
            sequence_length = shape[-2]
            position_embeddings = tf.convert_to_tensor(self.position_embeddings)
            position_embeddings = tf.slice(position_embeddings, (start_index, 0), (sequence_length, feature_length))
            print("Output shape/Positional Embedding:", position_embeddings.shape)
            return tf.broadcast_to(position_embeddings, shape)
    
        def compute_output_shape(self, input_shape):
            return input_shape

In [ ]:
with strategy.scope():
    class SequencePooling(layers.Layer):
        def __init__(self):
            super().__init__()
            self.attention = layers.Dense(1)
    
    def call(self, x):
        print("Input shape/Sequence Pooling:", x.shape)  # Debugging: print the shape of the inputs
        attention_weights = tf.nn.softmax(self.attention(x), axis=1)
        print("Attention weights shape/Sequence Pooling:", attention_weights.shape)  # Debugging: print the shape of the attention weights
        attention_weights = tf.transpose(attention_weights, perm=[0, 2, 1])
        weighted_representation = tf.matmul(attention_weights, x)
        print("Weighted representation shape/Sequence Pooling:", weighted_representation.shape)  # Debugging: print the shape of the weighted representation
        return tf.squeeze(weighted_representation, axis=-2)

In [ ]:
with strategy.scope():
    class StochasticDepth(layers.Layer):
        def __init__(self, drop_prop, **kwargs):
            super().__init__(**kwargs)
            self.drop_prob = drop_prop
    
        def call(self, x, training=None):
            if training and self.drop_prob > 0.0:
                keep_prob = 1 - self.drop_prob
                input_shape = tf.shape(x)
    
                # Debugging: Print the input shape
                print("StochasticDepth input shape:", x.shape)
    
                # Create a random tensor with the same shape as the input tensor
                random_tensor = keep_prob + tf.random.uniform(input_shape, dtype=x.dtype)
                binary_tensor = tf.floor(random_tensor)
    
                # Debugging: Print the shape of the binary tensor
                print("Binary tensor shape:", binary_tensor.shape)
    
                x = (x / keep_prob) * binary_tensor
    
                # Debugging: Print the output shape
                print("StochasticDepth output shape:", x.shape)
    
            return x

In [ ]:
with strategy.scope():
    def mlp(x, hidden_units, dropout_rate):
        for idx, units in enumerate(hidden_units):
            x = layers.Dense(units, activation=tf.nn.gelu)(x)
            # Debugging: Print the shape of the output after the Dense layer
            print(f"Layer {idx+1} - Dense/mlp: Output shape {x.shape}")
    
            x = layers.Dropout(dropout_rate)(x)
            # Debugging: Print the shape of the output after the Dropout layer
            print(f"Layer {idx+1} - Dropout/mlp: Output shape {x.shape}")
    
        return x

In [ ]:
with strategy.scope():
    def create_cct_model(
        image_size=image_size,
        input_shape=input_shape,
        num_heads=num_heads,
        projection_dim=projection_dim,
        transformer_units=transformer_units,
        transformer_layers=transformer_layers,
        positional_emb=True,
        stochastic_depth_rate=0.1,
    ):
        inputs = layers.Input(input_shape)
    
        # Tokenize input images
        cct_tokenizer = CCTTokenizer()
        encoded_patches = cct_tokenizer(inputs)
    
        # Debugging: Print the shape of encoded patches
        print("Encoded patches shape/create_cct_model:", encoded_patches.shape)
    
        # Apply positional embedding if enabled
        if positional_emb:
            sequence_length = encoded_patches.shape[1]
            encoded_patches += PositionEmbedding(sequence_length=sequence_length)(
                encoded_patches
            )
    
        # Calculate Stochastic Depth probabilities
        dpr = [x for x in np.linspace(0, stochastic_depth_rate, transformer_layers)]
    
        # Create multiple layers of the Transformer block
        for i in range(transformer_layers):
            # Layer normalization 1
            x1 = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
    
            # Multi-head attention
            attention_output = layers.MultiHeadAttention(
                num_heads=num_heads, key_dim=projection_dim, dropout=0.1
            )(x1, x1)
    
            # Debugging: Print the shape after multi-head attention
            print(f"Layer {i+1} - Multi-head attention output shape/create_cct_model: {attention_output.shape}")
    
            # Skip connection 1
            attention_output = StochasticDepth(dpr[i])(attention_output)
            x2 = layers.Add()([attention_output, encoded_patches])
    
            # Layer normalization 2
            x3 = layers.LayerNormalization(epsilon=1e-5)(x2)
    
            # MLP
            x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
    
            # Debugging: Print the shape after MLP
            print(f"Layer {i+1} - MLP output shape/create_cct_model: {x3.shape}")
    
            # Skip connection 2
            x3 = StochasticDepth(dpr[i])(x3)
            encoded_patches = layers.Add()([x3, x2])
    
        # Apply sequence pooling or global average pooling
        representation = layers.LayerNormalization(epsilon=1e-5)(encoded_patches)
        weighted_representation = SequencePooling()(representation)
        representation = layers.GlobalAveragePooling1D()(weighted_representation)  # Aggregate over tokens
    
        # Final classification layer
        logits = layers.Dense(1, activation='sigmoid')(representation)
    
        # Debugging: Print the shape of final output logits
        print("Final output logits shape/create_cct_model:", logits.shape)
    
        # Create the Keras model
        model = tf.keras.Model(inputs=inputs, outputs=logits)
        return model

In [ ]:
'''
def run_experiment(model):
    optimizer = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',  # Updated for binary classification
        metrics=[
            keras.metrics.BinaryAccuracy(name="accuracy"),  # Updated for binary classification
        ],
    )
    
    checkpoint_filepath = "/tmp/checkpoint.weights.h5"
    checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        train_dataset,
        epochs=num_epochs,
        validation_data=valid_dataset,
        class_weight=class_weights_dict,
        callbacks=[checkpoint_callback],
        verbose = 1,
    )


    # Uncomment and adjust the following code if you have a test dataset
    # model.load_weights(checkpoint_filepath)
    # _, accuracy = model.evaluate(x_test, y_test)
    # print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history


cct_model = create_cct_model()
history = run_experiment(cct_model)
'''

In [ ]:
from tensorflow.keras.metrics import BinaryAccuracy, AUC, Precision, Recall
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow import keras

with strategy.scope():
    def run_experiment(model):
        optimizer = keras.optimizers.Adam(learning_rate=0.001)
    
        model.compile(
            optimizer=optimizer,
            loss='binary_crossentropy',
            metrics=[
                BinaryAccuracy(name="accuracy"),
                AUC(name="auc"),
                Precision(name="precision"),
                Recall(name="recall"),
            ],
        )
        
        checkpoint_filepath = "D:/Models and Results/CCT_large.h5"
        checkpoint_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_filepath,
            monitor="val_auc",
            save_best_only=True,
            save_weights_only=True,
        )
    
        early_stopping_callback = EarlyStopping(
            monitor="val_loss",  # You can change this to "val_auc" or another metric
            patience=10,  # Number of epochs with no improvement after which training will be stopped
            restore_best_weights=True,  # Whether to restore model weights from the epoch with the best value of the monitored quantity
        )
    
        history = model.fit(
            train_dataset,
            epochs=num_epochs,
            validation_data=valid_dataset,
            class_weight=class_weights_dict,
            callbacks=[checkpoint_callback, early_stopping_callback],
            verbose=1,
        )
    
        # Load the best model weights
        # model.load_weights(checkpoint_filepath)
        
        # Evaluate the model on the test dataset
        test_metrics = model.evaluate(test_dataset, return_dict=True, verbose=1)
        
        # To calculate F1 Score, we need to get predictions for the test dataset
        y_pred = model.predict(test_dataset)
        # Assuming your model outputs probabilities for class 1 (MALIGNANT), and labels are binary
        y_pred_binary = np.round(y_pred).astype(int)  # Convert probabilities to binary predictions
        # Extract true labels from the test_dataset
        y_true = np.concatenate([y[1].numpy() for y in test_dataset])
        
        # Calculate F1 Score
        f1 = f1_score(y_true, y_pred_binary)
        test_metrics['f1_score'] = f1
        
        # Print all test metrics
        for metric, value in test_metrics.items():
            print(f"{metric}: {value}")
        
        return history

In [ ]:
with strategy.scope():
    cct_model = create_cct_model()
    history = run_experiment(cct_model)

In [ ]:
#Plotting Training and Validation Accuracy and Loss

# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
#Plotting Additional Metrics (e.g., AUC, Precision, Recall)
# Plot training & validation AUC
plt.figure(figsize=(12, 5))
plt.subplot(1, 3, 1)
plt.plot(history.history['auc'])
plt.plot(history.history['val_auc'])
plt.title('Model AUC')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plot training & validation Precision
plt.subplot(1, 3, 2)
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('Model Precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plot training & validation Recall
plt.subplot(1, 3, 3)
plt.plot(history.history['recall'])
plt.plot(history.history['val_recall'])
plt.title('Model Recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
# Assuming test_metrics is a dictionary containing the test results
test_metrics = cct_model.evaluate(test_dataset, return_dict=True)

# Print each metric
for metric, value in test_metrics.items():
    print(f"{metric}: {value}")

In [ ]:
# Assuming test_dataset is properly batched
y_true = np.concatenate([y for x, y in test_dataset], axis=0)
y_pred = cct_model.predict(test_dataset, verbose=1)


In [ ]:
from tensorflow.keras.metrics import AUC, Precision, Recall

auc = AUC()
precision = Precision()
recall = Recall()

# Update state with true labels and predicted probabilities
auc.update_state(y_true, y_pred)
precision.update_state(y_true, np.round(y_pred))
recall.update_state(y_true, np.round(y_pred))

# Finally, get the result
auc_result = auc.result().numpy()
precision_result = precision.result().numpy()
recall_result = recall.result().numpy()

print(f"AUC: {auc_result}")
print(f"Precision: {precision_result}")
print(f"Recall: {recall_result}")


In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Compute ROC curve and ROC area
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()


## DenseNet201

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
def build_densenet(NUM_CLASSES):
    inputs = layers.Input(shape=(image_size, image_size, 3))
    model = tf.keras.applications.densenet.DenseNet201(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(NUM_CLASSES, activation="sigmoid", name="pred")(x)

    model = tf.keras.Model(inputs, outputs, name="DenseNet201")
    return model

dense_model = build_densenet(NUM_CLASSES=1)


METRICS = [ 
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    tf.keras.metrics.AUC(name='prc', curve='PR'),
]


def scheduler(epoch, lr):
    if epoch % 10 == 0 and epoch != 0:
        return lr * 0.5
    return lr

learning_rate_scheduler = LearningRateScheduler(scheduler)

# Create Adam optimizer with weight decay.
optimizer = tf.keras.optimizers.Adam(
    learning_rate=1e-4)

# Compile the model.
dense_model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=METRICS,
)


# Create a learning rate scheduler callback.
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=7
)

# Create an early stopping callback.
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=10, restore_best_weights=True
)

# ModelCheckpoint callback to save the best model
checkpoint_filepath = 'D:/Models and Results/DenseNet201_thesis.h5'
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    monitor='val_loss',
    mode='min'
)

In [ ]:
# Fit the model.
dense_history = dense_model.fit(
    train_dataset,
    validation_data=valid_dataset, 
    class_weight=class_weights_dict,
    batch_size=batch_size,
    epochs=50,
    callbacks=[early_stopping, reduce_lr, model_checkpoint, learning_rate_scheduler],
    verbose=1,
)

In [ ]:
#Plotting Training and Validation Accuracy and Loss

# Plot training & validation accuracy values
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(dense_history.history['accuracy'])
plt.plot(dense_history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(dense_history.history['loss'])
plt.plot(dense_history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()

In [ ]:
#Plotting Additional Metrics (e.g., AUC, Precision, Recall)
# Plot training & validation AUC
plt.figure(figsize=(12, 5))
plt.subplot(1, 3, 1)
plt.plot(dense_history.history['auc'])
plt.plot(dense_history.history['val_auc'])
plt.title('Model AUC')
plt.ylabel('AUC')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plot training & validation Precision
plt.subplot(1, 3, 2)
plt.plot(dense_history.history['precision'])
plt.plot(dense_history.history['val_precision'])
plt.title('Model Precision')
plt.ylabel('Precision')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

# Plot training & validation Recall
plt.subplot(1, 3, 3)
plt.plot(dense_history.history['recall'])
plt.plot(dense_history.history['val_recall'])
plt.title('Model Recall')
plt.ylabel('Recall')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
# Assuming test_metrics is a dictionary containing the test results
test_metrics = dense_model.evaluate(test_dataset, return_dict=True)

# Print each metric
for metric, value in test_metrics.items():
    print(f"{metric}: {value}")

In [ ]:
# Assuming test_dataset is properly batched
y_true = np.concatenate([y for x, y in test_dataset], axis=0)
y_pred = dense_model.predict(test_dataset, verbose=1)

In [ ]:
from tensorflow.keras.metrics import AUC, Precision, Recall

auc = AUC()
precision = Precision()
recall = Recall()

# Update state with true labels and predicted probabilities
auc.update_state(y_true, y_pred)
precision.update_state(y_true, np.round(y_pred))
recall.update_state(y_true, np.round(y_pred))

# Finally, get the result
auc_result = auc.result().numpy()
precision_result = precision.result().numpy()
recall_result = recall.result().numpy()

print(f"AUC: {auc_result}")
print(f"Precision: {precision_result}")
print(f"Recall: {recall_result}")

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Compute ROC curve and ROC area
fpr, tpr, thresholds = roc_curve(y_true, y_pred)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
y_pred_binary = np.round(y_pred).astype(int)
f1 = f1_score(y_true, y_pred_binary)
test_metrics['f1_score'] = f1
print ('F1 score for DenseNet201: ',f1)